In [56]:
import os
import time
from datetime import timedelta

import cv2
import numpy as np
import streamlink
from imageai.Detection import VideoObjectDetection, ObjectDetection
# from apscheduler.schedulers.blocking import BlockingScheduler
from selenium import webdriver
import csv

def crop_frame(frame, target_img_name,y1=150,y2=500,x1=0,x2=1000):
    """
    only crop frame to evaluate the baseline. not a offical function, therefore outsied the class. will be deleted after evaluation.
    """
    frame = frame[y1:y2, x1:x2]
    
    if not os.path.isdir('.\\baseline'):
        os.makedirs('.\\baseline')
    path = '.\\baseline'
    cv2.imwrite(os.path.join(path , target_img_name), frame)
    return frame


london = 'https://www.earthcam.com/world/england/london/abbeyroad/?cam=abbeyroad_uk'
timesquare = 'https://www.earthcam.com/usa/newyork/timessquare/?cam=tsrobo1'
dublin = 'https://www.earthcam.com/world/ireland/dublin/?cam=templebar'

class CountingObject(object):
    """
    A class of counting objects
    """
    
    algos = {"resnet": "resnet50_coco_best_v2.0.1.h5", "yolov3": "yolo.h5", "yolo_tiny": "yolo-tiny.h5"}
    
    def __init__(self, stream_link):
        self.stream_link = stream_link
        self.streams = streamlink.streams(stream_link)
        if self.streams is None:
            raise ValueError("cannot open the stream link %s" % stream_link)
        
        #change 1, self detect quality
        q = list(self.streams.keys())[0]
        self.stream = self.streams['%s'%q]
        
        self.target_img_path = os.getcwd()
        
        self.detector = ObjectDetection()
        if self.detector is None:
            raise ValueError("Detector of objects is None")
        
        
    def detector_init(self, algo="resnet", speed="nomal"):
        """
        Must be invoked after instantiate for initialize a object detector. 
        
        Args:
            algo (str): The algorithm of object detection tasks. "resnet"(default), "yolov3", "yolo_tiny".
            speed (str): The detection speed for object detetion tasks. "normal"(default), "fast", "faster" , "fastest" and "flash".
        
        Returns:
            void
        
        """
        
        if algo == "resnet":
            self.detector.setModelTypeAsRetinaNet()
            self.detector.setModelPath(os.path.join(self.target_img_path, self.algos["resnet"]))
        elif algo == "yolov3":
            self.detector.setModelTypeAsYOLOv3()
            self.detector.setModelPath(os.path.join(self.target_img_path, self.algos["yolov3"]))
        elif algo == "yolo_tiny":
            self.detector.setModelTypeAsTinyYOLOv3()
            self.detector.setModelPath(os.path.join(self.target_img_path, self.algos["yolo_tiny"])) 
        else:
            print("Given algorithm of object detection is invalid.")
            return
        
        self.detector.loadModel(detection_speed=speed)
        self.custom_objects = self.detector.CustomObjects(person=True)

    def put_text_to_img(self, img, text, pos=(50,50), fontColor=(0,0,255), lineType=2):
        if img is None:
            print("Put text to a none image.")
            return
        
        font                  = cv2.FONT_HERSHEY_SIMPLEX
        fontScale             = 1

        cv2.putText(img, text,  
                    pos, 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
        
    def capture_frame_by_stream_wrapper(self, image_prefix="stream", mprob=30, num_im=6, time_interval=10):
        print("The current conuting function is based on capture frame by stream.")
        dir_path = '%s\%s'%(self.target_img_path,image_prefix)
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)
        frames_res = []
        if num_im <= 0:
            try:
                i = 0
                while True:
                    i = i + 1
                    frame_res = self.capture_frame_by_stream(image_prefix, i, mprob)
                    frames.res.append(frame_res)
                    time.sleep(time_interval)
            except KeyboardInterrupt:
                return frames_res
                print('Abort by key interrupt.')
        else:
            for i in range(num_im):
                frame_res = self.capture_frame_by_stream(image_prefix, i, mprob)
                frames_res.append(frame_res)
                time.sleep(time_interval)
            
            return frames_res
        
        

    def capture_frame_by_stream(self, image_prefix="stream", image_index=0, mprob=30)->int:
        video_cap = cv2.VideoCapture(self.stream.url)
        dir_path = os.path.join(self.target_img_path,image_prefix)
        if video_cap is None:
            print("Open webcam [%s] failed." % self.stream.url)
            return None
        else:
            ret, frame = video_cap.read()
            
            if not ret:
                print("Captured frame is broken.")
                video_cap.release()
                return None
            else:
                print("-----------------------------------------------------")
                
                print("Capturing frame %d." % image_index)
                target_img_name = "{}{}.png".format(image_prefix, image_index)
                frame = crop_frame(frame,target_img_name)# comment to unuse the crop function.

                detections = self.detector.detectCustomObjectsFromImage(custom_objects=self.custom_objects, 
                      input_type="array", 
                      input_image=frame, 
                      output_image_path=os.path.join(dir_path , target_img_name), 
                      minimum_percentage_probability=mprob)

                print("The number of person in frame %d (%s):" % (image_index, target_img_name), len(detections))
                print("The current time in frame %d (%s):" % (image_index, target_img_name), time.asctime())


                img = cv2.imread(os.path.join(dir_path , target_img_name))
                # put the number of persons to the image and put timestamp to the image
                self.put_text_to_img(img, "The number of person:%s " % str(len(detections)))
                self.put_text_to_img(img, "The current time:%s " % time.asctime(), pos=(50,300))

#               cv2.imshow("image", img)
                cv2.imwrite(os.path.join(dir_path , target_img_name), img)
#               cv2.waitKey(0) # blocked until pressing Enter key
#               cv2.destroyAllWindows()
                video_cap.release()
            
                return {'img_name':target_img_name, 'detected_num':len(detections)}
        
    def capture_frame_by_screenshot_wrapper(self, image_prefix="screenshot", mprob=30, num_im=6, time_interval=10):
        print("The current conuting function is based on capture frame by screenshot.")
        
        frames_res = []
        dir_path = os.path.join(self.target_img_path,image_prefix)
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)
        if num_im <= 0:
            try:
                i = 0
                while True:
                    i = i + 1
                    frame_res = self.capture_frame_by_screenshot(image_prefix, i, mprob)
                    frames_res.append(frame_res)
                    time.sleep(time_interval)
            except KeyboardInterrupt:
                if self.driver is not None:
                    self.driver.quit()
                return frames_res
                print('Abort by key interrupt.')
        else:
            for i in range(num_im):
                frame_res = self.capture_frame_by_screenshot(image_prefix, i, mprob)
                frames_res.append(frame_res)
                time.sleep(time_interval)
                
            if self.driver is not None:
                self.driver.quit()
            
            return frames_res
                
    def capture_frame_by_screenshot(self, image_prefix="screenshot", image_index=0, mprob=30, num_im=6)->int:
                
        if self.driver is None:
            print("Web driver is none.")
            return None
        else:
            print("-----------------------------------------------------")
            target_img_name = "{}{}.png".format(image_prefix, image_index)
            print("Taking screenshot %d..." % image_index)
            print("image path:%s" % os.path.join(self.target_img_path, target_img_name))
            self.driver.save_screenshot(os.path.join(self.target_img_path, target_img_name))
            detections = self.detector.detectCustomObjectsFromImage(custom_objects=self.custom_objects, 
                                              input_image=os.path.join(self.target_img_path, target_img_name), 
                                              output_image_path=os.path.join(dir_path, target_img_name), 
                                              minimum_percentage_probability=mprob)

            print("The number of person in frame %d (%s):" % (image_index, target_img_name), len(detections))
            print("The current time in frame %d (%s):" % (image_index, target_img_name), time.asctime())

            img = cv2.imread(os.path.join(dir_path, target_img_name))
            # put the number of persons to the image
            self.put_text_to_img(img, "The number of person is:%s" % str(len(detections)))
            self.put_text_to_img(img, "The current time:%s " % time.asctime(),pos=(50,300))

            cv2.imwrite(os.path.join(dir_path , target_img_name), img)
        
            return {'img_name':target_img_name, 'detected_num':len(detections)}

    def init_webdriver(self):
        self.driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
        self.driver.get(self.stream_link)
        time.sleep(15) # Jump over the ads
        
    def store_baseline_info_in_csv(self, info):
        if info is None:
            print("Given baseline data is None.")
            return
        baseline_info_path = os.path.join(self.target_img_path, "baseline.csv")
       
        with open(baseline_info_path, "w+") as csv_file:
            field_names = ["img_name", "countable_num", "detected_num"]
            csv_writer = csv.DictWriter(csv_file, fieldnames= field_names, restval=0)
            csv_writer.writeheader()
            for item in info:
                csv_writer.writerow(item)
                
            
if __name__ == "__main__":
#     scheduler = BlockingScheduler()
    print("Starting...")
    counting_person = CountingObject(dublin)
    counting_person.detector_init()

    by_stream_flag = True
    res = []
    if by_stream_flag:
        res = counting_person.capture_frame_by_stream_wrapper(image_prefix="dublin",num_im=2)
    else:
        counting_person.init_webdriver()
        res = counting_person.capture_frame_by_screenshot_wrapper(num_im=2)
    counting_person.store_baseline_info_in_csv(res)
    
    print('###Exit...')
        
#  raises errors, nor ready for using.
#     scheduler.add_job(capture_frame, 'interval', seconds=5, args=[cap, detect, custom_objects, target_img_path])
#     scheduler.start()



Starting...
The current conuting function is based on capture frame by stream.
-----------------------------------------------------
Capturing frame 0.
The number of person in frame 0 (dublin0.png): 6
The current time in frame 0 (dublin0.png): Wed Dec 12 21:36:43 2018
-----------------------------------------------------
Capturing frame 1.
The number of person in frame 1 (dublin1.png): 10
The current time in frame 1 (dublin1.png): Wed Dec 12 21:37:02 2018
###Exit...
